# **使用 Python 检查气候变化数据**

大家都知道，气候变化近来已成为一个巨大的话题。 有些人说这没有发生，而另一些人则说这是对地球生命的最大威胁。 我将检查 1700 年以来的温度数据，看看是否能在数据集中找到任何相关性。

In [ ]:
#导入Python数据分析库Pandas，并将其命名为pd。
#导入pandas库，用于数据处理和分析。
import pandas as pd

# **将数据集加载到dataframe**

In [ ]:
# 从CSV文件加载温度数据到DataFrame
temps_df = pd.read_csv('S:\GIT\PythonDataAnalysis\地表温度可视化数据集\GlobalLandTemperaturesByCountry.csv')

**检查dataframe**

In [ ]:
# 查看DataFrame的形状（行数和列数）
temps_df.shape

In [ ]:
# 清理数据，删除缺失值
temps_df.head()

# **描述数据**

This gives us a rough overview of the average temperature over the time period the data spans

In [ ]:
# 数据可视化，绘制温度趋势图
temps_df.AverageTemperature.describe()

# **检查缺失值**

In [ ]:
# 进行数据分析，计算统计指标  # 删除缺失值
temps_df[pd.isnull(temps_df.AverageTemperature)]

# **删除任何丢失的数据以清理数据集**

In [ ]:
#可视化温度趋势：

# 删除缺失值
temps_clean_df = temps_df.dropna()

In [ ]:
# 保存处理后的数据到新的CSV文件
temps_clean_df[pd.isnull(temps_clean_df.AverageTemperature)]

# **使用 Seaborn 绘制数据**

In [ ]:
# 其他数据处理和分析步骤
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
#设置图像大小为 600x600 像素
plt.figure(figsize=(60,60))
#绘制包含所有数据的线图
sns.lineplot(data=temps_clean_df)

# **导入工具来制作可以根据数据预测未来温度的模型**

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
#复制 temps_clean_df 数据框，以免修改原数据
temps_clean_df = temps_clean_df.copy()
#将日期列转换为日期时间格式
temps_clean_df['dt'] = pd.to_datetime(temps_clean_df['dt'])
#提取年份信息
temps_clean_df['Year'] = temps_clean_df['dt'].dt.year

# **分割dataframe**

In [ ]:
#选择年
X = temps_clean_df[['Year']]
# 选择目标变量
y = temps_clean_df['AverageTemperature']
# 拆分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# **添加模型/线性回归**

In [ ]:
# 创建线性回归模型
model = LinearRegression()
# 训练模型
model.fit(X_train, y_train)


# **预测未来气温**

In [ ]:
# 预测未来50年、100年、200年和500年的温度
# 获取数据中最大年份
last_year = temps_clean_df['Year'].max()
# 定义要预测的年份
years_ahead = [last_year + 50, last_year + 100, last_year + 200, last_year + 500]
# 创建未来年份数据框
future_years = pd.DataFrame({'Year': years_ahead})
 # 预测未来温度
predicted_temperatures = model.predict(future_years)

# **可视化数据**

In [ ]:
# 绘制观测数据和预测结果
plt.figure(figsize=(15, 6))
plt.scatter(temps_clean_df['Year'], temps_clean_df['AverageTemperature'], label='Observed Data')
plt.plot(future_years['Year'], predicted_temperatures, 'ro-', label='Predicted Temperatures')
plt.xlabel('Year')
plt.ylabel('Average Temperature')
plt.title('Simulation of Global Warming Effects')
plt.legend()
plt.grid(True)
plt.show()

# **评估模型的准确性**

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:
# 评估模型在测试集上的表现
y_pred = model.predict(X_test) # 预测测试集的温度

mse = mean_squared_error(y_test, y_pred) # 计算均方误差
rmse = np.sqrt(mse) # 计算均方根误差
mae = mean_absolute_error(y_test, y_pred)# 计算平均绝对误差
r2 = r2_score(y_test, y_pred)# 计算R方

print("Mean Squared Error:", mse)#print("均方误差:", mse)
print("Root Mean Squared Error:", rmse)#print("均方根误差:", rmse)
print("Mean Absolute Error:", mae)#print("平均绝对误差:", mae)
print("R-squared score:", r2)#print("R方:", r2)

# **可视化准确性**

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x=X_test['Year'], y=y_test, label='Actual Data')
sns.lineplot(x=future_years['Year'], y=predicted_temperatures, color='red', label='Predicted Temperatures')
plt.xlabel('Year')
plt.ylabel('Average Temperature')
plt.title('Simulation of Global Warming Effects')
plt.legend()
plt.grid(True)
plt.show()

# **使用 ElasticNet 评估/预测**

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

In [ ]:
# 数据标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# 超参数调优
param_grid = {
    'alpha': [0.87, 0.095, 0.0117],
    'l1_ratio': [0.1, 0.5, 1.0]
}

grid_search = GridSearchCV(ElasticNet(max_iter=1000), param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train)

best_alpha = grid_search.best_params_['alpha']
best_l1_ratio = grid_search.best_params_['l1_ratio']

print("Best alpha:", best_alpha)
print("Best l1_ratio:", best_l1_ratio)

In [ ]:
# 使用最佳超参数训练模型
model = ElasticNet(alpha=best_alpha, l1_ratio=best_l1_ratio, max_iter=1000)
model.fit(X_train_scaled, y_train)

In [ ]:
# 评估模型性能
y_pred = model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared score:", r2)

In [ ]:
# 预测未来温度
future_years = pd.DataFrame({'Year': np.arange(1750, 3000, 50)})
future_years_scaled = scaler.transform(future_years)
predicted_temperatures = model.predict(future_years_scaled)


In [ ]:
# 可视化
plt.figure(figsize=(15, 6))
sns.scatterplot(x=X['Year'], y=y, label='Historical Data')
sns.lineplot(x=future_years['Year'], y=predicted_temperatures, color='red', label='Predicted Temperatures (ElasticNet)')
plt.xlabel('Year')
plt.ylabel('Average Temperature')
plt.title('Simulation of Global Warming Effects (ElasticNet)')
plt.legend()
plt.grid(True)
plt.show()

# **结果**

根据预测，即使在大约 200 年内，我们也将达到极其危险的全球平均水平。 目前，全球平均气温约为 15 摄氏度（左右），200 年后，ML 模型预测我们可能会看到 +27.5 摄氏度（大约）的变化，500 年后，我们将处于 40 摄氏度左右。 全球平均摄氏温度..

# **此模拟的可能结果？**

人类健康：较高的气温可能导致与热有关的疾病和死亡。 热浪频率和强度的增加将对弱势群体构成重大风险，包括老年人、儿童和已有健康状况的人。

生态系统破坏：许多生态系统和物种适应特定的温度范围。 气温突然急剧升高可能会破坏这些生态系统，导致物种灭绝、生物多样性丧失以及自然食物链失衡。

农业和粮食安全：极端气温会对农业生产力产生负面影响。 农作物可能歉收，导致粮食短缺和价格飙升。 这反过来又可能加剧许多地区的饥饿和贫困。

水资源：气温升高会导致蒸发增加，导致干旱并影响水的可用性。 这将进一步导致人类消费、农业和工业的水资源紧张。

海平面上升：全球气温上升导致极地冰盖和冰川融化。 这将导致海平面大幅上升，威胁沿海地区，并使生活在低洼地区的数百万人流离失所。

极端天气事件：大气中更多的热量可能会加剧极端天气事件，例如飓风、台风和强烈风暴。 这些事件可能会造成大范围的破坏和人员伤亡。

海洋酸化：气温升高也会导致海洋酸化，这对海洋生物，特别是珊瑚礁和贝壳生物构成重大威胁。

全球气候模式的破坏：如此剧烈的温度变化可能会导致气候模式发生不可预测的变化，可能会扰乱季风、天气周期和区域气候，从而产生深远的影响。